<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/test_%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%97%86%EC%9D%B4_%EB%8B%A4%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import zipfile

# 1. Drive에서 로컬로 복사 (빠름)
shutil.copy("/content/drive/MyDrive/open.zip", "/content/open.zip")

# 2. 압축 풀기
with zipfile.ZipFile("/content/open.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")

# 3. 확인
!ls /content/train


컨티넨탈_10세대_2017_2019	     A_클래스_W177_2020_2025
어코드_10세대_2018_2022		     B_클래스_W246_2013_2018
1시리즈_F20_2013_2015		     뉴_스타일_코란도_C_2017_2019
1시리즈_F20_2016_2019		     프리우스_C_2018_2020
1시리즈_F40_2020_2024		     뉴_CC_2012_2016
그랜드카니발_2006_2010		     CLA_클래스_C117_2014_2019
2008_2015_2017			     CLA_클래스_C118_2020_2025
에쿠스_신형_2010_2015		     CLE_클래스_C236_2024_2025
파나메라_2010_2016		     CLS_클래스_C257_2019_2023
뉴_제타_2011_2016		     CLS_클래스_W218_2012_2017
뉴_카이엔_2011_2018		     아반떼_하이브리드_CN7_2021_2023
엑센트_신형_2011_2019		     아반떼_CN7_2021_2023
스파크_2012_2015		     더_뉴_아반떼_CN7_2023_2025
쿠퍼_컨트리맨_2012_2015		     CT6_2016_2018
올_뉴_모닝_2012_2015		     C_클래스_W204_2008_2015
말리부_2012_2016		     C_클래스_W205_2015_2021
아베오_2012_2016		     C_클래스_W206_2022_2024
뉴_티구안_2012_2016		     제네시스_DH_2014_2016
레이_2012_2017			     쏘나타_DN8_2020_2023
올란도_2012_2018		     쏘나타_디_엣지_DN8_2024_2025
더_뉴_파사트_2012_2019		     e_트론_2020_2023
트랙스_2013_2016		     E_PACE_2018_2020
콰트로포르테_2014_2016		     EQ900_2016_2018
더_뉴_아반떼_2014_2016		     E

In [2]:
import os
import glob
import torch
import timm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold


In [3]:
# train 경로
train_dir = '/content/train'

# test 경로
test_dir = '/content/test'

# sample_submission (추론 때 사용)
sample_submission_path = '/content/sample_submission.csv'


In [9]:
import torch
import gc

def show_memory_status():
    allocated = torch.cuda.memory_allocated() / (1024 ** 2)  # MB 단위
    reserved = torch.cuda.memory_reserved() / (1024 ** 2)    # MB 단위
    print(f"📊 현재 GPU 메모리 상태: Allocated = {allocated:.2f} MB | Reserved = {reserved:.2f} MB")

# 현재 CUDA 사용 가능 여부 확인
if torch.cuda.is_available():
    print("🔍 초기화 전 GPU 메모리 상태:")
    show_memory_status()

    # GPU 캐시 및 메모리 초기화
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

    print("\n🧹 GPU 메모리 초기화 완료")
    print("🔍 초기화 후 GPU 메모리 상태:")
    show_memory_status()
else:
    print("❌ CUDA 사용 불가")


🔍 초기화 전 GPU 메모리 상태:
📊 현재 GPU 메모리 상태: Allocated = 113.37 MB | Reserved = 136.00 MB

🧹 GPU 메모리 초기화 완료
🔍 초기화 후 GPU 메모리 상태:
📊 현재 GPU 메모리 상태: Allocated = 113.37 MB | Reserved = 136.00 MB


In [11]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os

class CarImageDataset(Dataset):
    def __init__(self, file_list, class_to_idx, transform=None):
        self.file_list = file_list
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]

        # 🚗 Step 1: Load Image (RGB 고정)
        image = Image.open(path).convert("RGB")

        # 🚗 Step 2: Apply Transform (Augmentation or Normalize)
        if self.transform:
            image = self.transform(image)

        # 🚗 Step 3: Extract label (CLASS_NAME)
        class_name = os.path.basename(os.path.dirname(path))
        label = self.class_to_idx[class_name]

        return image, label


In [12]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision import transforms
import glob
import os

# ✅ 전체 JPG 파일 불러오기 (Train) → 경로 수정!
file_list = glob.glob('/content/train/*/*.jpg')

# ✅ 클래스명 추출 (폴더명)
def extract_class_name_jpg(path):
    return os.path.basename(os.path.dirname(path))

class_names = sorted(set(extract_class_name_jpg(f) for f in file_list))
class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

print(f"✅ 클래스 수: {len(class_to_idx)}")  # 396개 나와야 정상

# ✅ 라벨 생성
labels = [class_to_idx[extract_class_name_jpg(f)] for f in file_list]

# ✅ Train/Val Split
train_files, val_files = train_test_split(file_list, test_size=0.1, stratify=labels, random_state=42)

# ✅ Transform 정의
train_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ Dataset 클래스
from PIL import Image
from torch.utils.data import Dataset

class CarJPGDataset(Dataset):
    def __init__(self, file_list, class_to_idx, transform=None):
        self.file_list = file_list
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        class_name = extract_class_name_jpg(path)
        label = self.class_to_idx[class_name]

        return image, label

# ✅ Dataset 정의
train_dataset = CarJPGDataset(train_files, class_to_idx, train_transform)
val_dataset = CarJPGDataset(val_files, class_to_idx, val_transform)

# ✅ DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)


✅ 클래스 수: 396


In [13]:
import timm
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model('efficientnet_b5', pretrained=True, num_classes=396)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

# AdamW + weight_decay 추가 추천 (EffNet 계열에 많이 사용)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)


In [ ]:
from sklearn.model_selection import StratifiedKFold
import copy
import torch
from tqdm import tqdm
import timm
import glob
import os

# ✅ device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 전체 jpg 파일
file_list = glob.glob('/content/train/*/*.jpg')

# ✅ 클래스명 추출
def extract_class_name_jpg(path):
    return os.path.basename(os.path.dirname(path))

class_names = sorted(set(extract_class_name_jpg(f) for f in file_list))
class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

print(f"✅ 클래스 수: {len(class_to_idx)}")

# ✅ 라벨 생성
labels = [class_to_idx[extract_class_name_jpg(f)] for f in file_list]

# ✅ transform 정의
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

train_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ Dataset 클래스 정의 (JPG용)
class CarJPGDataset(Dataset):
    def __init__(self, file_list, class_to_idx, transform=None):
        self.file_list = file_list
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        class_name = extract_class_name_jpg(path)
        label = self.class_to_idx[class_name]

        return image, label

# ✅ StratifiedKFold 정의
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ✅ 5-Fold 루프 시작
for fold, (train_idx, val_idx) in enumerate(skf.split(file_list, labels)):
    print(f"\n==============================")
    print(f"🔁 Fold {fold + 1} / 5")
    print(f"==============================\n")

    # ✅ Fold별 split
    train_files = [file_list[i] for i in train_idx]
    val_files = [file_list[i] for i in val_idx]

    # ✅ Fold별 Dataset & DataLoader
    train_dataset = CarJPGDataset(train_files, class_to_idx, train_transform)
    val_dataset = CarJPGDataset(val_files, class_to_idx, val_transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)

    # ✅ Fold별 model / criterion / optimizer 초기화
    model = timm.create_model('efficientnet_b5', pretrained=True, num_classes=396)
    model = model.to(device)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

    # ✅ EarlyStopping 변수 초기화
    best_val_loss = float('inf')
    patience = 3
    patience_counter = 0
    best_model_wts = copy.deepcopy(model.state_dict())

    # ✅ Epoch 루프
    for epoch in range(1, 31):
        print(f"\n📌 Fold {fold+1} | Epoch {epoch}")

        # === 학습 ===
        model.train()
        train_loss = 0.0
        train_correct = 0

        loop = tqdm(train_loader, desc=f"Train Fold {fold+1}", leave=False)
        for X, y in loop:
            X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * X.size(0)
            train_correct += (outputs.argmax(1) == y).sum().item()
            loop.set_postfix(loss=loss.item())

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct / len(train_loader.dataset)

        # === 검증 ===
        model.eval()
        val_loss = 0.0
        val_correct = 0

        val_loop = tqdm(val_loader, desc=f"Valid Fold {fold+1}", leave=False)
        with torch.no_grad():
            for X, y in val_loop:
                X, y = X.to(device, non_blocking=True), y.to(device, non_blocking=True)
                outputs = model(X)
                loss = criterion(outputs, y)
                val_loss += loss.item() * X.size(0)
                val_correct += (outputs.argmax(1) == y).sum().item()
                val_loop.set_postfix(loss=loss.item())

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        # === 로그 출력 ===
        print(f"✅ Fold {fold+1} | Epoch {epoch} | Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
        print(f"✅ Fold {fold+1} | Epoch {epoch} | Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

        # === EarlyStopping ===
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            save_path = f"/content/drive/MyDrive/team_models/EffNetB5_fold{fold+1}.pth"
            torch.save(model.state_dict(), save_path)
            print(f"📦 Best model saved for Fold {fold+1}!")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"⚠️ EarlyStopping patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("⛔ Early stopping triggered.")
                break

    # ✅ Fold 끝나고 Best 모델 로드
    model.load_state_dict(best_model_wts)
    print(f"✅ Fold {fold+1} Best model loaded.\n")


✅ 클래스 수: 396

🔁 Fold 1 / 5


📌 Fold 1 | Epoch 1


KeyboardInterrupt: 

In [ ]:
# ✅ 앙상블 Inference 예시 (5개 모델 평균)

import os
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import timm
from torchvision import transforms

# 설정
FOLD_MODEL_PATHS = [
    "/content/drive/MyDrive/team_models/EffNetB5_fold1.pth",
    "/content/drive/MyDrive/team_models/EffNetB5_fold2.pth",
    "/content/drive/MyDrive/team_models/EffNetB5_fold3.pth",
    "/content/drive/MyDrive/team_models/EffNetB5_fold4.pth",
    "/content/drive/MyDrive/team_models/EffNetB5_fold5.pth",
]

TEST_DIR = "/content/drive/MyDrive/open/clean_test/test"
SAMPLE_SUB_PATH = "/content/drive/MyDrive/open/sample_submission.csv"
SAVE_SUBMISSION_PATH = "/content/drive/MyDrive/team_models/submission_fold5_ensemble.csv"
NUM_CLASSES = 396

# 샘플 제출 파일에서 클래스명 추출
sample = pd.read_csv(SAMPLE_SUB_PATH)
column_names = sample.columns.tolist()[1:]  # 'ID' 제외

# ✅ Transform
transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ 테스트용 Dataset
class TestNPYDataset(Dataset):
    def __init__(self, npy_root, transform=None):
        self.file_list = []
        for root, dirs, files in os.walk(npy_root):
            for file in files:
                if file.endswith('.npy'):
                    self.file_list.append(os.path.join(root, file))
        self.file_list.sort()

        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        image = np.load(path, mmap_mode='r').copy()
        image = torch.from_numpy(image).float()
        image = image.repeat(3, 1, 1)
        image = image.contiguous()

        if self.transform:
            image = self.transform(image)

        fname = os.path.basename(path).replace(".npy", "")
        return image, fname

# ✅ DataLoader
test_dataset = TestNPYDataset(TEST_DIR, transform)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=6,
    pin_memory=True,
    prefetch_factor=4
)

# ✅ 앙상블 결과 초기화
ensemble_outputs = []

# ✅ 각 Fold 모델 순서대로 추론
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold_idx, model_path in enumerate(FOLD_MODEL_PATHS):
    print(f"\n🚀 Inference with Fold {fold_idx + 1} Model: {model_path}")

    # 모델 로드
    model = timm.create_model('efficientnet_b5', pretrained=False, num_classes=NUM_CLASSES)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # fold별 output 저장
    fold_probs = []

    with torch.no_grad():
        for imgs, names in tqdm(test_loader, desc=f"🔍 Fold {fold_idx + 1} Inference"):
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)
            fold_probs.append(probs.cpu().numpy())

    fold_probs = np.concatenate(fold_probs, axis=0)
    ensemble_outputs.append(fold_probs)

# ✅ 앙상블 평균
ensemble_outputs = np.stack(ensemble_outputs, axis=0)  # (num_folds, num_samples, num_classes)
mean_outputs = np.mean(ensemble_outputs, axis=0)       # (num_samples, num_classes)

# ✅ 결과 저장
results = []
for idx, path in enumerate(test_dataset.file_list):
    fname = os.path.basename(path).replace(".npy", "")
    row = {"ID": fname}
    row.update({class_name: mean_outputs[idx, i] for i, class_name in enumerate(column_names)})
    results.append(row)

submission_df = pd.DataFrame(results)
submission_df = submission_df[["ID"] + column_names]
submission_df.to_csv(SAVE_SUBMISSION_PATH, index=False)

print(f"\n✅ 앙상블 서브미션 저장 완료: {SAVE_SUBMISSION_PATH}")



🚀 Inference with Fold 1 Model: /content/drive/MyDrive/team_models/EffNetB5_fold1.pth


🔍 Fold 1 Inference: 100%|██████████| 130/130 [08:20<00:00,  3.85s/it]



🚀 Inference with Fold 2 Model: /content/drive/MyDrive/team_models/EffNetB5_fold2.pth


🔍 Fold 2 Inference: 100%|██████████| 130/130 [00:33<00:00,  3.91it/s]



🚀 Inference with Fold 3 Model: /content/drive/MyDrive/team_models/EffNetB5_fold3.pth


...


🚀 Inference with Fold 4 Model: /content/drive/MyDrive/team_models/EffNetB5_fold4.pth


🔍 Fold 4 Inference: 100%|██████████| 130/130 [00:32<00:00,  4.05it/s]



🚀 Inference with Fold 5 Model: /content/drive/MyDrive/team_models/EffNetB5_fold5.pth


🔍 Fold 5 Inference: 100%|██████████| 130/130 [00:32<00:00,  4.00it/s]



✅ 앙상블 서브미션 저장 완료: /content/drive/MyDrive/team_models/submission_fold5_ensemble.csv


In [ ]:
from google.colab import files

# 다운로드할 경로 지정 (네가 저장한 경로 그대로 사용!)
files.download('/content/drive/MyDrive/team_models/submission_fold5_ensemble.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 앙상블이랑 TTA 적용한 것
import os
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import timm
from torchvision import transforms

# 설정
FOLD_MODEL_PATHS = [
    "/content/drive/MyDrive/team_models/EffNetB5_fold1.pth",
    "/content/drive/MyDrive/team_models/EffNetB5_fold2.pth",
    "/content/drive/MyDrive/team_models/EffNetB5_fold3.pth",
    "/content/drive/MyDrive/team_models/EffNetB5_fold4.pth",
    "/content/drive/MyDrive/team_models/EffNetB5_fold5.pth",
]

TEST_DIR = "/content/drive/MyDrive/open/clean_test/test"
SAMPLE_SUB_PATH = "/content/drive/MyDrive/open/sample_submission.csv"
SAVE_SUBMISSION_PATH = "/content/drive/MyDrive/team_models/submission_fold5_tta.csv"
NUM_CLASSES = 396
TTA_TIMES = 3   # ✅ TTA 반복 횟수 (가볍게 3~5 추천)

# 샘플 제출 파일에서 클래스명 추출
sample = pd.read_csv(SAMPLE_SUB_PATH)
column_names = sample.columns.tolist()[1:]  # 'ID' 제외

# ✅ Transform 정의 (여기서 TTA용 transform을 리스트로 만들 예정)
# 기본 Normalize
base_transform = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])

# ✅ TTA용 transform list
tta_transforms = [
    transforms.Compose([base_transform]),  # 원본
    transforms.Compose([
        transforms.RandomHorizontalFlip(p=1.0),  # Flip
        base_transform
    ]),
    transforms.Compose([
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
        base_transform
    ]),
    # 필요시 더 추가 가능 (주의: 과하면 성능 저하)
]

# ✅ 테스트용 Dataset
class TestNPYDataset(Dataset):
    def __init__(self, npy_root):
        self.file_list = []
        for root, dirs, files in os.walk(npy_root):
            for file in files:
                if file.endswith('.npy'):
                    self.file_list.append(os.path.join(root, file))
        self.file_list.sort()

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        image = np.load(path, mmap_mode='r').copy()
        image = torch.from_numpy(image).float()
        image = image.repeat(3, 1, 1)
        image = image.contiguous()

        fname = os.path.basename(path).replace(".npy", "")
        return image, fname

# ✅ DataLoader 고정
test_dataset = TestNPYDataset(TEST_DIR)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=6,
    pin_memory=True,
    prefetch_factor=4
)

# ✅ 전체 Fold 앙상블 결과 초기화
ensemble_outputs = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Fold별 모델 반복
for fold_idx, model_path in enumerate(FOLD_MODEL_PATHS):
    print(f"\n🚀 Inference with Fold {fold_idx + 1} Model: {model_path}")

    # 모델 로드
    model = timm.create_model('efficientnet_b5', pretrained=False, num_classes=NUM_CLASSES)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Fold별 output 저장
    fold_probs = []

    # ✅ TTA 반복
    for tta_idx in range(TTA_TIMES):
        print(f"📢 TTA Round {tta_idx + 1} / {TTA_TIMES} for Fold {fold_idx + 1}")

        # TTA transform 선택 (순환)
        tta_transform = tta_transforms[tta_idx % len(tta_transforms)]

        # fold + TTA round별 확률 저장
        tta_probs = []

        with torch.no_grad():
            for imgs, names in tqdm(test_loader, desc=f"🔍 Fold {fold_idx + 1} | TTA {tta_idx + 1}"):
                imgs = imgs.to(device)
                # ✅ transform 적용
                imgs = torch.stack([tta_transform(img) for img in imgs])

                outputs = model(imgs)
                probs = F.softmax(outputs, dim=1)
                tta_probs.append(probs.cpu().numpy())

        # TTA 결과 저장
        tta_probs = np.concatenate(tta_probs, axis=0)
        fold_probs.append(tta_probs)

    # ✅ Fold 내 TTA 평균
    fold_probs = np.stack(fold_probs, axis=0)      # (tta_times, num_samples, num_classes)
    fold_mean_probs = np.mean(fold_probs, axis=0)  # (num_samples, num_classes)

    # ✅ Fold 결과 → ensemble list 추가
    ensemble_outputs.append(fold_mean_probs)

# ✅ Fold 앙상블 평균
ensemble_outputs = np.stack(ensemble_outputs, axis=0)  # (num_folds, num_samples, num_classes)
mean_outputs = np.mean(ensemble_outputs, axis=0)       # (num_samples, num_classes)

# ✅ 최종 결과 저장
results = []
for idx, path in enumerate(test_dataset.file_list):
    fname = os.path.basename(path).replace(".npy", "")
    row = {"ID": fname}
    row.update({class_name: mean_outputs[idx, i] for i, class_name in enumerate(column_names)})
    results.append(row)

submission_df = pd.DataFrame(results)
submission_df = submission_df[["ID"] + column_names]
submission_df.to_csv(SAVE_SUBMISSION_PATH, index=False)

print(f"\n✅ Fold 앙상블 + TTA 서브미션 저장 완료: {SAVE_SUBMISSION_PATH}")



🚀 Inference with Fold 1 Model: /content/drive/MyDrive/team_models/EffNetB5_fold1.pth
📢 TTA Round 1 / 3 for Fold 1


🔍 Fold 1 | TTA 1:  18%|█▊        | 24/130 [00:55<04:06,  2.33s/it]


KeyboardInterrupt: 